We used the ribosome profiling of *E. coli* from Mohammad et al. (2019) to train the neural network model. This requires iXnos (https://github.com/lareaulab/iXnos). The scripts align.sh and trim_linker.pl were modified to process Mohammad's data.

```console
# align.sh
# bowtie-build
# rsem-prepare-reference

GENOME_DIR=$1
PROC_DIR=$2

for i in `awk 'BEGIN{FS="/"} NR==1 {print $NF}' sra.id`; do \
	fasterq-dump ${i} -e 24
done

if [ ! -f $PROC_DIR/trimmed.fastq ]; then
        cat $PROC_DIR/*.fastq | $PROC_DIR/trim_linker.pl > $PROC_DIR/trimmed.fastq
fi

if [ ! -f $GENOME_DIR/ecol.ncrna.1.ebwt ]; then
	bowtie-build $GENOME_DIR/ecol.ncrna.fa $GENOME_DIR/ecol.ncrna
fi

if [ ! -f $GENOME_DIR/ecol.transcripts.13cds10.1.ebwt ]; then
        bowtie-build $GENOME_DIR/ecol.transcripts.13cds10.fa $GENOME_DIR/ecol.transcripts.13cds10
fi

if [ ! -f $GENOME_DIR/ecol.transcripts.13cds10.idx.fa ]; then
	rsem-prepare-reference $GENOME_DIR/ecol.transcripts.13cds10.fa $GENOME_DIR/ecol.transcripts.13cds10
fi

if [ ! -f $PROC_DIR/not_ncrna.fastq ]; then
	bowtie -v 2 -p 36 -S --un $PROC_DIR/not_ncrna.fastq \
        $GENOME_DIR/ecol.ncrna \
        $PROC_DIR/trimmed.fastq > $PROC_DIR/ncrna.sam 2> $PROC_DIR/ncrna.bowtiestats
fi

if [ ! -f $PROC_DIR/footprints.sam ]; then
bowtie -a --norc -v 2 -p 36 -S --un $PROC_DIR/unmapped.fastq \
       $GENOME_DIR/ecol.transcripts.13cds10 \
       $PROC_DIR/not_ncrna.fastq > $PROC_DIR/footprints.sam 2> $PROC_DIR/footprints.bowtiestats
fi

rsem-calculate-expression --sam $PROC_DIR/footprints.sam $GENOME_DIR/ecol.transcripts.13cds10 $PROC_DIR/rsem 2> $PROC_DIR/rsem.stderr

samtools view -h $PROC_DIR/rsem.transcript.bam > $PROC_DIR/rsem.transcript.sam
```

```console
# trim_linker.pl 
#!/usr/bin/perl
use strict;

# Trim off end sequence matching the linker from Mohammad's experiments:
# CTGTAGGCACCATCAAT... (it can be longer than this)

my $printstring = "";

while(<>)
{
    my $header;
    my $seq;
    my $qheader;
    my $qual;
    
    if (/^@/)
    { 
	print $printstring;
	$printstring = "";

	$header = $_; chomp $header;
	$seq = <>; chomp $seq;
	$qheader = <>; chomp $qheader;
	$qual = <>; chomp $qual;
    }

    my $trimmed;

    # no random barcodes at beginning
    if ( ( ($trimmed) = $seq =~ /^([ACGTN]+)CTGTAGGCACCATCAAT/ ) or
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)CTGTAGGCACCATCAA$/ ) or
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)CTGTAGGCACCATCA$/ ) or
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)CTGTAGGCACCATC$/ ) or
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)CTGTAGGCACCAT$/ ) or
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)CTGTAGGCACCA$/ ) or
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)CTGTAGGCACC$/ ) or
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)CTGTAGGCAC$/ ) or
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)CTGTAGGCA$/ ) or
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)CTGTAGGC$/ ) or
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)CTGTAGG$/ ) or
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)CTGTAG$/ ) or
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)CTGTA$/ ) or
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)CTGT$/ ) or
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)CTG$/ ) or
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)CT$/ ) or
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)C$/ ) or ## ...eats up anything ending in a C... but that's ok
	 ( ($trimmed) = $seq =~ /^([ACGTN]+)$/ )  ## ...no evidence of linker..
	)
    {
    
	my $length = length($trimmed);

	if ($length > 10)
	{
	    $printstring = $header . "\n" . $trimmed . "\n" . $qheader . "\n";
	    $printstring .= substr( $qual, 0, $length ); ## fix this bug in other versions of the script!!
	    $printstring .= "\n";
	}
    }
    
    print $printstring;
    $printstring = "";
}
```

We first prepared *E. coli* transcript files.

```console
# Get all CDS
zcat Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.42.gtf.gz \
| gtfToGenePred stdin stdout | awk '$6!=$7' | genePredToBed stdin ecol.bed
bedtools getfasta -fi Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.dna.toplevel.fa -bed ecol.bed -fo ecol.transcripts.txt -s -split -name -tab
sed -i 's/(+)//;s/(-)//' ecol.transcripts.txt
# Get padding of 13 and 10 at the 5' and 3' UTRs
# Remove frameshift ORF AAC75929 (prfB)
zcat Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.42.gtf.gz \
| gtfToGenePred stdin stdout | awk '$6!=$7' | genePredToBed stdin stdout \
| awk 'BEGIN{OFS="\t"} $10==1 {if($6~/+/){print $1,$2-13,$3+10,$4,$5,$6} 
else{print $1,$2-10,$3+13,$4,$5,$6}}' > ecol.13cds10.bed
bedtools getfasta -fi Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.dna.toplevel.fa -bed ecol.13cds10.bed -fo ecol.transcripts.13cds10.txt -s -name -tab
sed -i 's/(+)//;s/(-)//' ecol.transcripts.13cds10.txt

# Get padding of 50 and 10 at the 5' and 3' UTRs for opening energy calculation
# Remove frameshift ORF AAC75929 (prfB)
zcat Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.42.gtf.gz \
| gtfToGenePred stdin stdout | awk '$6!=$7' | genePredToBed stdin stdout \
| awk 'BEGIN{OFS="\t"} $10==1 {if($6~/+/){print $1,$2-50,$3+10,$4,$5,$6} 
else{print $1,$2-10,$3+50,$4,$5,$6}}' > ecol.50cds10.bed
bedtools getfasta \
-fi Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.dna.toplevel.fa \
-bed ecol.50cds10.bed \
-s -name -tab \
| sed 's/(+)//;s/(-)//' > ecol.transcripts.50cds10.txt
join -t$'\t' \
<(sort -k1,1 tx2gene.txt) \
<(sort -k1,1 ecol.transcripts.50cds10.txt) \
| awk '{print ">" $2 "\n" $3}' > ecol.transcripts.50cds10.fa

# Get transcripts with ATG GTG TTG start codons
ecol.transcripts.txt
join -t$'\t' \
<(awk '{print $2,$1}' ecol.transcripts.txt | awk '/^ATG|^GTG|^TTG/ {print $2}' | sort) \
<(sort -k1,1 ecol.transcripts.13cds10.txt) \
| awk '{print ">" $1 "\n" $2}' > ecol.transcripts.13cds10.fa
awk 'BEGIN{RS=">"}NR>1{sub("\n","\t"); gsub("\n",""); print $0}' ecol.transcripts.13cds10.fa \
| awk 'BEGIN{OFS="\t"}{print $1,"13",length($2),"10"}' > ecol.transcripts.13cds10.lengths.txt
```

We downloaded and examine the individual sra files. SRR7759806 and SRR7759807 showed the best triplet periodicity. They were merged for subsequent analysis.

```console
# Download sra files
mkdir ~/src/iXnos/expts/mohammad
cd ~/src/iXnos/expts/mohammad
wget -O SRP158945_metadata.csv 'http://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?save=efetch&db=sra&rettype=runinfo&term=SRP158945'
sed 's/,/\t/g' SRP158945_metadata.csv | cut -f1,11 \
| awk '$1~/SRR/ {print "/sra/sra-instant/reads/ByRun/sra/SRR/SRR775/" $1 "/" $1 ".sra"}' > sra.id
ascp -l640M -T -i ~/.aspera/connect/etc/asperaweb_id_dsa.openssh --user=anonftp --host=ftp.ncbi.nlm.nih.gov --mode=recv --file-list=sra.id .

# Align reads
for i in *.fastq; do \
  cd ~/src/iXnos/expts
  mkdir ${i%%.*}
  mv $i ${i%%.*}
  cp align.sh trim_linker.pl ${i%%.*}
  cd ${i%%.*}
  ./align.sh ~/src/iXnos/genome_data/ .
done

# Examine plots
for i in SRR7759805 SRR7759808 SRR7759809 SRR7759810 SRR7759811 SRR7759812 SRR7759813 SRR7759814; do \
  cd ~/src/iXnos/expts/mohammad;
  mv $i process;
  python ~/anaconda2/pkgs/iXnos/plot_frames.py;
  mv process $i;
  mv plots plots_$i;
done

# Merge SRR7759806 and SRR7759807
cd ~/src/iXnos/expts/mohammad
mkdir process
samtools merge process/footprints.sam \
SRR7759806/footprints.sam \
SRR7759807/footprints.sam
cd process
rsem-calculate-expression --sam footprints.sam ~/src/iXnos/genome_data/ecol.transcripts.13cds10 rsem 2> rsem.stderr
samtools view -h rsem.transcript.bam > rsem.transcript.sam
python ~/anaconda2/pkgs/iXnos/plot_frames.py
```

We then train a neural network model using the processed ribosome profiling data

In [1]:
import numpy as np
import lasagne as ls
import iXnos.interface as inter
import iXnos.optimizecodons as opt
import iXnos.process as proc

In [2]:
expt_dir = "/home/chunshenlim/src/iXnos/expts/mohammad/"
sam_fname = expt_dir + "/process/rsem.transcript.sam"
inter.edit_sam_file(
    expt_dir, sam_fname, filter_unmapped=True, 
    sam_add_simple_map_wts=False, RSEM_add_map_wts=True)

'/home/chunshenlim/src/iXnos/expts/mohammad//process/rsem.transcript.mapped.wts.sam'

In [3]:
expt_dir = "/home/chunshenlim/src/iXnos/expts/mohammad/"
sam_fname = expt_dir + "/process/rsem.transcript.mapped.wts.sam"
gene_len_fname = "/home/chunshenlim/src/iXnos/genome_data/ecol.transcripts.13cds10.lengths.txt"
inter.do_frame_and_size_analysis(
    expt_dir, sam_fname, gene_len_fname, min_size=13, 
    max_size=36, verbose=False)

In [4]:
expt_dir = "/home/chunshenlim/src/iXnos/expts/mohammad/"
sam_fname = expt_dir + "/process/rsem.transcript.mapped.wts.sam"
gene_len_fname = "/home/chunshenlim/src/iXnos/genome_data/ecol.transcripts.13cds10.lengths.txt"
# A site offset rules
shift_dict = {25:{0:12, 1:False, 2:False}}
# Number of codons to ignore at either end of CDS in model training
cod_trunc_5p = 20
cod_trunc_3p = 20
# Minimum and maximum allowed footprint sizes
min_fp_size = 25
max_fp_size = 25
# Choose number of genes in training and test sets
num_tr_genes = 333
num_te_genes = 167
# Set coverage cutoffs for genes in training/test sets
# Minimum footprint counts per gene
min_cts_per_gene = 200
# Minimum A site codons with nonzero counts
min_cod_w_data = 100
# Transcriptome file, e.g. for yeast
gene_seq_fname = "/home/chunshenlim/src/iXnos/genome_data/ecol.transcripts.13cds10.fa"

In [5]:
inter.process_sam_file(
    expt_dir, sam_fname, gene_seq_fname, gene_len_fname,
    shift_dict, cod_trunc_5p, cod_trunc_3p, min_fp_size,
    max_fp_size, num_tr_genes, num_te_genes, min_cts_per_gene,
    min_cod_w_data, paralog_groups_fname=False,
    overwrite=True)

file /home/chunshenlim/src/iXnos/expts/mohammad//process/cts_by_codon.size.25.25.txt already exists
file /home/chunshenlim/src/iXnos/expts/mohammad//process/outputs.size.25.25.txt already exists
making file /home/chunshenlim/src/iXnos/expts/mohammad//process/tr_set_bounds.size.25.25.trunc.20.20.min_cts.200.min_cod.100.top.500.txt
making file /home/chunshenlim/src/iXnos/expts/mohammad//process/te_set_bounds.size.25.25.trunc.20.20.min_cts.200.min_cod.100.top.500.txt
610 genes meeting data cutoff


In [6]:
name = "high_salt"
expt_dir = "/home/chunshenlim/src/iXnos/expts/mohammad/"
outputs_fname = expt_dir + "/process/outputs.size.25.25.txt"
gene_seq_fname = "/home/chunshenlim/src/iXnos/genome_data/ecol.transcripts.13cds10.fa"
gene_len_fname = "/home/chunshenlim/src/iXnos/genome_data/ecol.transcripts.13cds10.lengths.txt"
tr_codons_fname = expt_dir + "/process/tr_set_bounds.size.25.25.trunc.20.20.min_cts.200.min_cod.100.top.500.txt"
te_codons_fname = expt_dir + "/process/te_set_bounds.size.25.25.trunc.20.20.min_cts.200.min_cod.100.top.500.txt"
rel_cod_idxs = range(-3,3)
rel_nt_idxs = [cod_idx * 3 + i for cod_idx in rel_cod_idxs for i in range(3)]
widths = [200]
nonlinearity = "tanh"
update_method = "nesterov"

In [7]:
# Create neural network
neural_net = inter.make_lasagne_feedforward_nn(
    name, expt_dir, gene_seq_fname, gene_len_fname, tr_codons_fname,
    te_codons_fname, outputs_fname, rel_cod_idxs=rel_cod_idxs,
    rel_nt_idxs=rel_nt_idxs, nonlinearity=nonlinearity, widths=widths, 
    update_method=update_method)

Making dense mlp
nonnegative model


In [8]:
# Run neural network for a desired Number of epochs
num_epochs = 100
neural_net.run_epochs(num_epochs)

learning rate: 0.01
Epoch 1 took 4.049s
  training loss:		6.721418
  test loss:		6.753441
learning rate: 0.00941176470588
Epoch 2 took 3.730s
  training loss:		6.584628
  test loss:		6.733057
learning rate: 0.00888888888889
Epoch 3 took 3.662s
  training loss:		6.558858
  test loss:		6.786018
learning rate: 0.00842105263158
Epoch 4 took 3.472s
  training loss:		6.562880
  test loss:		6.683421
learning rate: 0.008
Epoch 5 took 3.565s
  training loss:		6.534845
  test loss:		6.669208
learning rate: 0.00761904761905
Epoch 6 took 3.391s
  training loss:		6.526851
  test loss:		6.671323
learning rate: 0.00727272727273
Epoch 7 took 3.328s
  training loss:		6.513343
  test loss:		6.639606
learning rate: 0.00695652173913
Epoch 8 took 3.707s
  training loss:		6.481373
  test loss:		6.625272
learning rate: 0.00666666666667
Epoch 9 took 3.471s
  training loss:		6.458457
  test loss:		6.679230
learning rate: 0.0064
Epoch 10 took 3.591s
  training loss:		6.439775
  test loss:		6.598577
learning rat

learning rate: 0.00164948453608
Epoch 82 took 3.691s
  training loss:		5.466751
  test loss:		7.245365
learning rate: 0.00163265306122
Epoch 83 took 3.721s
  training loss:		5.460148
  test loss:		7.213522
learning rate: 0.00161616161616
Epoch 84 took 3.909s
  training loss:		5.453848
  test loss:		7.203413
learning rate: 0.0016
Epoch 85 took 3.633s
  training loss:		5.437642
  test loss:		7.160859
learning rate: 0.00158415841584
Epoch 86 took 3.981s
  training loss:		5.431930
  test loss:		7.318815
learning rate: 0.00156862745098
Epoch 87 took 3.712s
  training loss:		5.416323
  test loss:		7.207446
learning rate: 0.00155339805825
Epoch 88 took 4.012s
  training loss:		5.399868
  test loss:		7.195398
learning rate: 0.00153846153846
Epoch 89 took 3.886s
  training loss:		5.385329
  test loss:		7.284057
learning rate: 0.00152380952381
Epoch 90 took 3.878s
  training loss:		5.357571
  test loss:		7.320951
learning rate: 0.00150943396226
Epoch 91 took 3.979s
  training loss:		5.364010
  t

In [9]:
RANDOM_SEED = 12345
RANDOM = np.random.RandomState(RANDOM_SEED)
ls.random.set_rng(RANDOM)
nn_dir = "/home/chunshenlim/src/iXnos/expts/mohammad/lasagne_nn/high_salt/"
epoch = 19 # lowest test loss
nt_feats = True
my_nn = inter.load_lasagne_feedforward_nn(nn_dir, epoch)

# Test for scoring
cod_seq = "ATGGCCTGTGATGAATTTGGTCACATTAAACTCAACCCTCAACGCTCCACTGTCTGGTATTAG"
rel_cod_idxs = range(-3,3)
rel_nt_idxs = [cod_idx * 3 + i for cod_idx in rel_cod_idxs for i in range(3)]
cds_score = opt.score_cod_seq_full(cod_seq, my_nn, rel_cod_idxs, rel_nt_idxs)

Making dense mlp
nonnegative model


In [10]:
cds_score

17.00791147385095

```console
# Score the PSI:Biolog targets
python ~/avoidance/src/Avoidance-v2/Random_forest/iXnos.py \
-i pET21_NESG.fa \
-d ~/avoidance/src/iXnos/expts/mohammad/lasagne_nn/high_salt \
-e 19 -p 32
```